<div align=center>

# Principles of Simulation:<br />Final Project

By Hamed Araab & Shahriar Khalvati

</div>


### Prerequisites


In [ ]:
import random
import numpy as np
import pandas as pd
import seaborn as sns

from framework import *

### Visualzier
💡 In oreder to run again, restart the notebook to remove variables.

In [1]:
import threading
import eel

def run_eel():
    eel.init("Visualizer")


    @eel.expose  # Expose this function to Javascript
    def say_hello_py(x):
        print("Hello from %s" % x)


    # eel.say_hello_js("Python World!")  # Call a Javascript function

    eel.start("index.html", mode=None)  # Start (this blocks and enters loop)

# Run eel in a separate thread
eel_thread = threading.Thread(target=run_eel)
eel_thread.start()

# Continue with other cells in the notebook
print("Eel server is running in a separate thread.")

Eel server is running in a separate thread.


In [5]:
eel.initVisualizer(2,4)

<function eel._call_return.<locals>.return_func(callback: Optional[Callable[..., Any]] = None, error_callback: Optional[Callable[..., Any]] = None) -> Any>

### Servers


In [ ]:
class Doctor:
    def __init__(self, controller) -> None:
        self.controller = controller
        self.status: Literal["available", "busy"] = "available"


class Examiner(Doctor):
    pass


class Physician(Doctor):
    pass

In [ ]:
class DoctorSelectionStrategy:
    @staticmethod
    def randomAvailable(doctors: List[Doctor]) -> Doctor:
        selectedDoctor: Doctor | None = None

        random.shuffle(doctors)

        for doctor in doctors:
            if doctor.status == "available":
                selectedDoctor = doctor

                break

        return selectedDoctor

### Patients


In [ ]:
class Patient:
    pass


class HighPriorityPatient(Patient):
    pass


class LowPriorityPatient(Patient):
    pass

### Controller


In [ ]:
class HospitalController(SimController):
    def __init__(self, examinersCount=1, physiciansCount=1) -> None:
        super().__init__(stopTime=32 * 24 * 60, initialEvent=ArrivalEvent(initial=True))

        self.treatedPatients: List[Patient] = []
        self.highPriorityExaminationQueue: List[HighPriorityPatient] = []
        self.lowPriorityExaminationQueue: List[LowPriorityPatient] = []
        self.treatmentQueue: List[Patient] = []

        self.examiners: List[Examiner] = [
            Examiner(controller=self) for i in range(1, examinersCount + 1)
        ]

        self.physicians: List[Physician] = [
            Physician(controller=self) for i in range(1, physiciansCount + 1)
        ]

### Events


#### Arrival


In [ ]:
ARRIVAL_EVENT_INTERVAL: Callable[[], float] | None = None


class ArrivalEvent(SimEvent[HospitalController]):
    def __init__(self, initial: bool = False) -> None:
        super().__init__(interval=0 if initial else ARRIVAL_EVENT_INTERVAL())

        self.patient = (
            LowPriorityPatient() if random.random() > 0.15 else HighPriorityPatient()
        )

    def trigger(self) -> None:
        self.controller.dispatchEvent(ArrivalEvent())

        appropriateQueue = (
            self.controller.highPriorityExaminationQueue
            if self.patient is HighPriorityPatient
            else self.controller.lowPriorityExaminationQueue
        )

        selectedExaminer = DoctorSelectionStrategy.randomAvailable(
            self.controller.examiners
        )

        if appropriateQueue or not selectedExaminer:
            appropriateQueue.append(self.patient)
        else:
            selectedExaminer.status = "busy"

            self.controller.dispatchEvent(
                ExaminationFinishEvent(patient=self.patient, examiner=selectedExaminer)
            )

#### Examination Finish


In [ ]:
EXAMINATION_FINISH_EVENT_INTERVAL: Callable[[Patient], float] | None = None


class ExaminationFinishEvent(SimEvent[HospitalController]):
    def __init__(self, patient: Patient, examiner: Examiner) -> None:
        super().__init__(interval=EXAMINATION_FINISH_EVENT_INTERVAL(patient))

        self.patient = patient
        self.examiner = examiner

    def trigger(self) -> None:
        selectedPhysician = DoctorSelectionStrategy.randomAvailable(
            self.controller.physicians
        )

        if self.controller.treatmentQueue or not selectedPhysician:
            self.controller.treatmentQueue.append(self.patient)
        else:
            selectedPhysician.status = "busy"

            self.controller.dispatchEvent(
                TreatmentFinishEvent(patient=self.patient, physician=selectedPhysician)
            )

        if self.controller.highPriorityExaminationQueue:
            patient = self.controller.highPriorityExaminationQueue.pop(0)
        elif self.controller.lowPriorityExaminationQueue:
            patient = self.controller.lowPriorityExaminationQueue.pop(0)
        else:
            patient = None

        if patient:
            self.controller.dispatchEvent(
                ExaminationFinishEvent(patient=patient, examiner=self.examiner)
            )
        else:
            self.examiner.status = "available"

#### Treatment Finish


In [ ]:
TREATMENT_FINISH_EVENT_INTERVAL: Callable[[Patient], float] | None = None


class TreatmentFinishEvent(SimEvent[HospitalController]):
    def __init__(self, patient: Patient, physician: Physician) -> None:
        super().__init__(interval=TREATMENT_FINISH_EVENT_INTERVAL(patient))

        self.patient = patient
        self.physician = physician

    def trigger(self) -> None:
        self.controller.treatedPatients.append(self.patient)

        patient = (
            self.controller.treatmentQueue.pop(0)
            if self.controller.treatmentQueue
            else None
        )

        if patient:
            self.controller.dispatchEvent(
                TreatmentFinishEvent(patient=patient, physician=self.physician)
            )
        else:
            self.physician.status = "available"

In [ ]:
def positivieRandomNormal(mu, sigma):
    num = np.random.normal(mu, sigma)

    if num > 0:
        return num

    return positivieRandomNormal(mu, sigma)

In [ ]:
ARRIVAL_EVENT_INTERVAL = lambda: positivieRandomNormal(20, 4)

EXAMINATION_FINISH_EVENT_INTERVAL = lambda patient: (
    positivieRandomNormal(30, 5)
    if patient is HighPriorityPatient
    else positivieRandomNormal(15, 10)
)

TREATMENT_FINISH_EVENT_INTERVAL = lambda patient: (
    positivieRandomNormal(28, 13)
    if patient is HighPriorityPatient
    else positivieRandomNormal(10, 7)
)